# Moltbook Karma Data Engineering Pipeline

**Objetivo**: Pipeline de ingenieria de datos end-to-end para predecir el karma de usuarios en moltbook.com

**Variable Target**: `users.karma` (regresion)

[![Python 3.10+](https://img.shields.io/badge/python-3.10+-blue.svg)](https://www.python.org/downloads/)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)

![Moltbook Banner](../assets/banner.jpg)

Moltbook.com es una red social donde agentes de IA interactúan entre sí de forma autónoma: publican, comentan y votan. Lo interesante es que a veces discuten temas sensibles como vulnerabilidades de sistemas, lo cual genera preocupación sobre cómo estos bots pueden corromperse entre ellos.

Este proyecto nace de la necesidad de analizar el **karma** (la reputación de cada agente) para entender qué factores determinan la influencia dentro de la plataforma. Los hallazgos pueden servir para estrategias de seguridad y moderación en sistemas de IA.

Esta red social esta compuesta por:

**Entidades**:
- `users`: Perfiles de agentes
- `posts`: Publicaciones en comunidades
- `comments`: Comentarios en posts
- `sub_molt`: Comunidades tematicas

**Variable Objetivo**: 
- `karma` : puntuación de reputación del usuario

## 1. Configuracion del Entorno

En primer lugar configuramos el entorno de trabajo, importando las librerías y configurando el setup necesario para para el desarrollo del proyecto.

### 1.1 Imports y Setup

Se importan las librerías necesarias, se configura el path y se crean los directorios de trabajo.

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import logging
import polars as pl
from config.settings import settings

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
settings.ensure_directories()
print(f"Project root: {settings.project_root}")
print(f"Data directory: {settings.data_dir}")

Project root: c:\Users\Paulina Peralta\Desktop\moltbook-karma
Data directory: c:\Users\Paulina Peralta\Desktop\moltbook-karma\data


## 2. Web Scraping con Playwright

Playwright es 

### 2.1 Inicializacion de Base de Datos

Para ello en primer lugar inicializamos la base de datos mediante la función init_database() y definimos un db_ops para manejar las operaciones con la base de datos.

In [2]:
from src.database.connection import init_database, check_database_exists
from src.database.operations import DatabaseOperations
from src.database.models import User, Post, Comment, SubMolt

if not check_database_exists():
    print("Initializing database...")
    init_database()
else:
    print("Database already exists")

db_ops = DatabaseOperations()
print(f"Current counts - Users: {db_ops.count(User)}, Posts: {db_ops.count(Post)}")

Database already exists
Current counts - Users: 981, Posts: 1242


### 2.2 Ejecucion del Scraper

Luego ejecutamos el scraper mediante el método scrape_all, que es el orquestador principal. Su función es ejecutar el flujo completo de recolección de datos: el pipeline scrapea usuarios, submolts y posts, y genera un reporte. ¿Cómo lo hace? Primero busca nuevas URLs de usuarios en la lista global /u, respetando el límite de max_users. Luego procesa cada usuario: entra en cada perfil, descarga el HTML (o utiliza el caché) y extrae datos como karma, nombre, entre otros. Finalmente, guarda y actualiza esta información en la base de datos.

Luego, para scrapear submolts, busca nuevas comunidades en la lista global /m, respetando el límite de max_submolts. Se procesa cada submolt entrando a la página de la comunidad y guardando sus datos. Para extraer los posts dentro de la página de cada submolt, lo que hace es buscar y extraer todos los posts viables en esa página, y los guarda en la base de datos asociados a ese submolt y a su autor. Finalmente, genera un reporte consultando la base de datos para contar cuántos posts y comentarios totales hay guardados, y devuelve un diccionario con el resumen: { "users": X, "submolts": Y, "posts": Z, "comments": W }.

## 3. Exploratory Data Analysis

Para el análisis exploratorio de los datos utilizamos polars, que es una librería que introduce el concepto de lazyframes, que a diferencia de pandas, que carga y procesa todo en memoria inmediatamente, polars solo lo construye en un plan de ejecución.

Permitiendo optimización automática, ya que utilizando .collect() polars analiza todo el plan y lo optimiza antes de la ejecución, y puede combinar filtros eliminando pasos redundantes, leer solo las columnas necesarias, y está diseñado para usar todos los núcleos de la CPU en paralelo.

Y al ser lazy, con polars podemos procesar el dataset y cargar los datos en bloques, procesar, y liberar la memoria.


### 3.1 Carga de Datos con Polars

Para cargar los datos utilizamos polars, donde creamos un **LazyFrame** que lo que hace es solo crear como un plan de acción o un mapa de dónde están los datos, en este caso de users y posts, donde no gasta tanta memoria, lo cual es ideal para trabajar con muchos datos.

Luego, con .collect() se da el orden de ejecución, donde ponemos `users_lf.collect()`. Ahí polars se va a la base de datos, lee la información y la convierte en un dataframe listo para usar.


In [ ]:
from src.processing.silver import load_table_to_lazy
users_lf = load_table_to_lazy("users")
posts_lf = load_table_to_lazy("posts")
comments_lf = load_table_to_lazy("comments")
users_df = users_lf.collect()
posts_df = posts_lf.collect()
print(f"Users: {len(users_df)} records")
print(f"Posts: {len(posts_df)} records")

2026-02-09 16:44:38,147 - INFO - Loaded 981 rows from users
2026-02-09 16:44:38,167 - INFO - Loaded 1242 rows from posts
2026-02-09 16:44:38,178 - INFO - Loaded 644 rows from comments


Users: 981 records
Posts: 1242 records


### 3.2 Estadísticas Descriptivas de Karma

Para poder entender como el Karma esta distribuido imprimimos sus estadísticas y vemos su distribución.

In [ ]:
karma_stats = users_df.select([
    pl.col("karma").mean().alias("mean"),
    pl.col("karma").median().alias("median"),
    pl.col("karma").std().alias("std"),
    pl.col("karma").min().alias("min"),
    pl.col("karma").max().alias("max"),
    pl.col("karma").quantile(0.25).alias("q25"),
    pl.col("karma").quantile(0.75).alias("q75"),
])
print("Karma Statistics:")
print(karma_stats)

Karma Statistics:
shape: (1, 7)
┌─────────────┬────────┬──────────────┬─────┬────────┬─────┬─────┐
│ mean        ┆ median ┆ std          ┆ min ┆ max    ┆ q25 ┆ q75 │
│ ---         ┆ ---    ┆ ---          ┆ --- ┆ ---    ┆ --- ┆ --- │
│ f64         ┆ f64    ┆ f64          ┆ i64 ┆ i64    ┆ f64 ┆ f64 │
╞═════════════╪════════╪══════════════╪═════╪════════╪═════╪═════╡
│ 6553.063201 ┆ 0.0    ┆ 44460.967544 ┆ 0   ┆ 500002 ┆ 0.0 ┆ 0.0 │
└─────────────┴────────┴──────────────┴─────┴────────┴─────┴─────┘


Se visualiza que la distribución está muy sesgada con mediana igual a cero y un máximo mayor a 500K. Donde mayoría de usuarios tienen karma bajo y unos pocos acumulan valores extremos.

### 3.3 Distribución de Variables

Luego visualizamos la distribución de las variables categóricas.

In [ ]:
user_stats = users_df.select([
    pl.col("followers").mean().alias("avg_followers"),
    pl.col("following").mean().alias("avg_following"),
    pl.col("description").is_not_null().sum().alias("users_with_description"),
    pl.col("human_owner").is_not_null().sum().alias("users_with_owner"),
])

print("User Profile Statistics:")
print(user_stats)

User Profile Statistics:
shape: (1, 4)
┌───────────────┬───────────────┬────────────────────────┬──────────────────┐
│ avg_followers ┆ avg_following ┆ users_with_description ┆ users_with_owner │
│ ---           ┆ ---           ┆ ---                    ┆ ---              │
│ f64           ┆ f64           ┆ u32                    ┆ u32              │
╞═══════════════╪═══════════════╪════════════════════════╪══════════════════╡
│ 0.462793      ┆ 0.046891      ┆ 41                     ┆ 41               │
└───────────────┴───────────────┴────────────────────────┴──────────────────┘


## 4. Limpieza y Preparacion de Datos

### 4.1 Pipeline de Limpieza con Polars Lazy

Creamos una función build_silver_layer() que lo que hace es tomar los datos crudos que vienen de nuestra base de datos, donde se limpian y se estandarizan los datos, convirtiendo fechas, arreglando textos y asegurando los tipos de datos correctos. Luego almacenamos en parquet, que exporta estos datos limpios a archivos .parquet en la carpeta data/silver.

In [ ]:
from src.processing.silver import build_silver_layer

silver_results = build_silver_layer()
print("Silver Layer Built:")
for table, count in silver_results.items():
    print(f"  {table}: {count} records")

silver_users = pl.read_parquet(settings.silver_dir / "users.parquet")
print(f"\nSilver users sample:")
print(silver_users.head(3))

2026-02-09 16:44:47,539 - INFO - Loaded 981 rows from users
2026-02-09 16:44:47,619 - INFO - Wrote 981 users to c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\silver\users.parquet
2026-02-09 16:44:47,639 - INFO - Loaded 1242 rows from posts
2026-02-09 16:44:47,671 - INFO - Wrote 1242 posts to c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\silver\posts.parquet
2026-02-09 16:44:47,682 - INFO - Loaded 644 rows from comments
2026-02-09 16:44:47,692 - INFO - Wrote 644 comments to c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\silver\comments.parquet
2026-02-09 16:44:47,700 - INFO - Loaded 55 rows from sub_molt
2026-02-09 16:44:47,706 - INFO - Wrote 55 submolts to c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\silver\submolts.parquet
2026-02-09 16:44:47,709 - INFO - Silver layer build complete: {'users': 981, 'posts': 1242, 'comments': 644, 'submolts': 55}


Silver Layer Built:
  users: 981 records
  posts: 1242 records
  comments: 644 records
  submolts: 55 records

Silver users sample:
shape: (3, 9)
┌─────────────┬─────────────┬───────┬────────────┬───┬────────┬───────────┬───────────┬────────────┐
│ id_user     ┆ name        ┆ karma ┆ descriptio ┆ … ┆ joined ┆ followers ┆ following ┆ scraped_at │
│ ---         ┆ ---         ┆ ---   ┆ n          ┆   ┆ ---    ┆ ---       ┆ ---       ┆ ---        │
│ str         ┆ str         ┆ i64   ┆ ---        ┆   ┆ str    ┆ i64       ┆ i64       ┆ str        │
│             ┆             ┆       ┆ str        ┆   ┆        ┆           ┆           ┆            │
╞═════════════╪═════════════╪═══════╪════════════╪═══╪════════╪═══════════╪═══════════╪════════════╡
│ user_7ca81d ┆ AureliusPro ┆ 0     ┆            ┆ … ┆ null   ┆ 0         ┆ 0         ┆ 2026-02-09 │
│ d06237      ┆ tocol       ┆       ┆            ┆   ┆        ┆           ┆           ┆ T19:05:22. │
│             ┆             ┆       ┆         

Aqui se visualizan los resultados de la limpieza, y se confirma que se limpio correctamente, donde se genero una base de datos analitica y limpia para ser utilizada.

## 5. Feature Engineering (Gold Layer)

### 5.1 Ingenieria de Features con Polars Lazy

Se realizo el uso de Polars debido a que nos encontramos con un dataset enorme . Para esto se realiza la carga del gold layer. En esta layer transformaremos datos limpios en features

In [10]:
from src.processing.gold import build_gold_layer

gold_results = build_gold_layer()
print("Gold Layer Built:")
print(f"  User features: {gold_results['user_features']} records")
print(f"  Feature columns: {gold_results['feature_columns']}")

2026-02-09 16:44:52,150 - INFO - Loaded users from silver layer
2026-02-09 16:44:52,152 - INFO - Loaded posts from silver layer
2026-02-09 16:44:52,152 - INFO - Loaded comments from silver layer
2026-02-09 16:44:52,154 - INFO - Loaded submolts from silver layer
2026-02-09 16:44:52,237 - INFO - Wrote 981 user feature records with 21 columns to c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\gold\user_features.parquet
2026-02-09 16:44:52,239 - INFO - Feature columns: ['id_user', 'name', 'karma', 'followers', 'following', 'follower_ratio', 'has_description', 'has_human_owner', 'description_length', 'post_count', 'total_post_rating', 'avg_post_rating', 'max_post_rating', 'avg_title_length', 'avg_post_desc_length', 'comment_count', 'total_comment_rating', 'avg_comment_rating', 'avg_comment_length', 'total_activity', 'total_rating']
2026-02-09 16:44:52,239 - INFO - Gold layer build complete: {'user_features': 981, 'feature_columns': 21}


Gold Layer Built:
  User features: 981 records
  Feature columns: 21


### 5.2 Descripcion de Features

Las features creadas son:

- **Perfil**: followers, following, follower_ratio (métricas de red social)
- **Contenido**: has_description, description_length (qué tan completo está el perfil)
- **Actividad en Posts**: post_count, avg_post_rating, total_post_rating
- **Actividad en Comentarios**: comment_count, avg_comment_rating
- **Agregadas**: total_activity, total_rating (métricas combinadas)

In [11]:
from src.processing.gold import get_modeling_data

features_df = get_modeling_data()
print("Feature Columns:")
for col in features_df.columns:
    dtype = features_df[col].dtype
    print(f"  {col}: {dtype}")
print(f"\nFeature Statistics:")
print(features_df.describe())

Feature Columns:
  id_user: String
  name: String
  karma: Int64
  followers: Int64
  following: Int64
  follower_ratio: Float64
  has_description: Int32
  has_human_owner: Int32
  description_length: UInt32
  post_count: UInt32
  total_post_rating: Int64
  avg_post_rating: Float64
  max_post_rating: Int64
  avg_title_length: Float64
  avg_post_desc_length: Float64
  comment_count: UInt32
  total_comment_rating: Int64
  avg_comment_rating: Float64
  avg_comment_length: Float64
  total_activity: UInt32
  total_rating: Int64

Feature Statistics:
shape: (9, 22)
┌────────────┬────────────┬──────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ statistic  ┆ id_user    ┆ name ┆ karma      ┆ … ┆ avg_commen ┆ avg_comme ┆ total_act ┆ total_rat │
│ ---        ┆ ---        ┆ ---  ┆ ---        ┆   ┆ t_rating   ┆ nt_length ┆ ivity     ┆ ing       │
│ str        ┆ str        ┆ str  ┆ f64        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆            ┆    

Se observa que hay mucha variabilidad en los datos :
Se obtuvieron 981 registros, 20 columnas

## 6. Modelado con H2O AutoML

### 6.1 Entrenamiento del Modelo

Se utiliza **H2O AutoML** porque automatiza la selección y optimización de modelos. Entrena varios algoritmos (GBM, Random Forest, Deep Learning, GLM) y elige el mejor según la métrica objetivo.

In [14]:
from src.models.trainer import H2OTrainer, FEATURE_COLUMNS

trainer = H2OTrainer(max_models=10,max_runtime_secs=300,)
print("Training H2O AutoML model...")
print(f"Features: {FEATURE_COLUMNS}")
results = trainer.train(data=features_df,target="karma",features=FEATURE_COLUMNS,)
print(f"\nBest Model: {results['model_id']}")

Training H2O AutoML model...
Features: ['followers', 'following', 'follower_ratio', 'has_description', 'has_human_owner', 'description_length', 'post_count', 'total_post_rating', 'avg_post_rating', 'max_post_rating', 'avg_title_length', 'avg_post_desc_length', 'comment_count', 'total_comment_rating', 'avg_comment_rating', 'avg_comment_length', 'total_activity', 'total_rating']
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.461-b11, mixed mode)
  Starting server from C:\Users\Paulina Peralta\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\PAULIN~1\AppData\Local\Temp\tmppivf28ph
  JVM stdout: C:\Users\PAULIN~1\AppData\Local\Temp\tmppivf28ph\h2o_Paulina_Peralta_started_from_python.out
  JVM stderr: C:\Users\PAULIN~1\AppData\Local\Temp\tmppivf28ph\h2o_Paulina_Peralta_started_from_python.err
  Server is running at http://127.0.0.1:5432

H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,-03:00
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_Paulina_Peralta_lt8mxu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.549 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


2026-02-09 16:46:20,011 - INFO - H2O initialized
2026-02-09 16:46:20,011 - INFO - Training with 18 features, 981 samples


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


2026-02-09 16:46:21,670 - INFO - Train size: 773, Test size: 208


AutoML progress: |
16:46:21.780: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:46:21.799: AutoML: XGBoost is not available; skipping it.
16:46:21.869: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]


16:46:22.530: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]

█
16:46:23.693: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]

█
16:46:24.122: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]

█
16:46:24.630: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]
16:46:24.933: _train param, Dropping bad and constant columns: [avg_comment_rating, total_comment_rating]



2026-02-09 16:46:28,869 - INFO - Best model: GBM_2_AutoML_1_20260209_164621
2026-02-09 16:46:28,889 - INFO - Training complete - MAE: 3234.8072, RMSE: 23050.9321, R2: 0.6363



Best Model: GBM_2_AutoML_1_20260209_164621


Se concluye que el best model es el GBM_2_AutoML. Esto quiere decir que es la version 2 del modelo GBM, el algoritmo usado es el gradient boosting y la mejora es que ayuda a corregir el error (real - preddicion ) del primer modelo


### 6.2 Evaluacion del Modelo

Se calculan los indicadores para poder evaluar el modelo.

In [15]:
print("Model Evaluation Metrics:")
print(f"  MAE:  {results['mae']:.4f}")
print(f"  RMSE: {results['rmse']:.4f}")
print(f"  R2:   {results['r2']:.4f}")
print(f"\n  Train samples: {results['train_size']}")
print(f"  Test samples:  {results['test_size']}")

Model Evaluation Metrics:
  MAE:  3234.8072
  RMSE: 23050.9321
  R2:   0.6363

  Train samples: 773
  Test samples:  208


Donde:

- **R² = 0.64**: el modelo explica el 64% de la varianza del karma. 
- **MAE = 3,235**: en promedio se equivoca por ~3K puntos de karma.
- **RMSE alto**: significa que hay outliers extremos en la variable objetivo. Los usuarios que tienen un Karma muy alto.

### 6.3 Predicciones

Se realizan las predicciones del modelo.

In [16]:
predictions = trainer.predict(features_df)
comparison = predictions.select(["name", "karma", "karma_predicted"]).head(10)
print("Actual vs Predicted Karma:")
print(comparison)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Actual vs Predicted Karma:
shape: (10, 3)
┌──────────────────┬───────┬─────────────────┐
│ name             ┆ karma ┆ karma_predicted │
│ ---              ┆ ---   ┆ ---             │
│ str              ┆ i64   ┆ f64             │
╞══════════════════╪═══════╪═════════════════╡
│ AureliusProtocol ┆ 0     ┆ 176.243253      │
│ ZenithGarcia     ┆ 0     ┆ 176.243253      │
│ NimbusDrifts     ┆ 0     ┆ 176.243253      │
│ MBC20MintPoster  ┆ 0     ┆ 176.243253      │
│ DivineLuna       ┆ 0     ┆ 176.243253      │
│ Clawd            ┆ 0     ┆ 176.243253      │
│ NebulaBot2026    ┆ 0     ┆ 176.243253      │
│ Virgil_DT        ┆ 0     ┆ 176.243253      │
│ Shellraiser      ┆ 0     ┆ 176.243253      │
│ PenkoAI          ┆ 0     ┆ 176.243253      │
└──────────────────┴───────┴─────────────────┘


c:\Users\Paulina Peralta\anaconda3\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Se almacena el modelo entrenado.

In [17]:
model_path = trainer.save_model()
print(f"Model saved to: {model_path}")
pred_path = settings.models_dir / "predictions.parquet"
predictions.write_parquet(pred_path)
print(f"Predictions saved to: {pred_path}")

2026-02-09 16:46:54,142 - INFO - Saved model to C:\Users\Paulina Peralta\Desktop\moltbook-karma\data\models\GBM_2_AutoML_1_20260209_164621


Model saved to: C:\Users\Paulina Peralta\Desktop\moltbook-karma\data\models\GBM_2_AutoML_1_20260209_164621
Predictions saved to: c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\models\predictions.parquet


## 7. Conclusiones

### 7.1 Resumen del Pipeline

In [18]:
print("=" * 50)
print("RESUMEN DEL PIPELINE")
print("=" * 50)
print(f"\n1. Web Scraping:")
print(f"   - Usuarios: {db_ops.count(User)}")
print(f"   - Posts: {db_ops.count(Post)}")
print(f"   - SubMolts: {db_ops.count(SubMolt)}")
print(f"\n2. Procesamiento:")
print(f"   - Silver layer: {settings.silver_dir}")
print(f"   - Gold layer: {settings.gold_dir}")
print(f"\n3. Modelado:")
print(f"   - Algoritmo: H2O AutoML")
print(f"   - Target: karma (regresion)")
print(f"   - MAE: {results.get('mae', 'N/A'):.4f}")
print(f"   - R2: {results.get('r2', 'N/A'):.4f}")
print("\n" + "=" * 50)

RESUMEN DEL PIPELINE

1. Web Scraping:
   - Usuarios: 981
   - Posts: 1242
   - SubMolts: 55

2. Procesamiento:
   - Silver layer: c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\silver
   - Gold layer: c:\Users\Paulina Peralta\Desktop\moltbook-karma\data\gold

3. Modelado:
   - Algoritmo: H2O AutoML
   - Target: karma (regresion)
   - MAE: 3234.8072
   - R2: 0.6363



* H20 AutoML pudo elegir un algoritmo optimo
* Las features engineered relevantes fueron follower_ratio, total_activity,
* El modelo llega a ser un buen predictor debido a la elecciondel AutoMl puesto que mas 63% de la varianza se encuentra explicada por el modelo
* El MAE salio alto (3234.8) Esto quiere decir que el modelo predice ±3234 puntos de karma de diferencia. Esto tambien puede ser ya que se extrajo moltobook user con bastante karma como nuevos, por lo que se podria mejorar cuando se extraiga mas usuarios

Por todo ello podemos decir que la relacion usuario con el karma puede llegar a ser predecible

### 7.2 Observaciones

1. **Web Scraping**: Se realizo el web scrapping en un proyecto modular en dodne se dividio por clases cada parte como usuarios, en donde se uso las herramientas como beautiful soup, se considero :
•⁠  ⁠Hacer un webscrapping para usuarios nuevos y usuarios con alto karma, es por ello que se observa una gran variabilidad en ellos
•⁠  ⁠Hacer un webscrapping para submolt, en donde en ecada submolt se encuentran los post mas rankeados y dentro de ellos los comentarios mas rankeados. Por temas del scope del trabajo se determino un limite por cada estructura de datos anidada.

2. **Procesamiento con Polars**: Lazy evaluation optimiza memoria y rendimiento

3. **Feature Engineering**: Las features derivadas (follower_ratio, total_activity) capturan engagement

4. **Modelado**: H2O AutoML automatiza la seleccion del mejor algoritmo

5. **Limitaciones**: 
   - El dataset es pequeno para produccion
   - Algunas features pueden tener alta correlacion
   - El karma puede depender de factores no capturados (tiempo en plataforma, calidad de contenido)